In [ ]:
import os
import logging
import openai
import openai
from openai import OpenAI


os.environ["OPENAI_API_BASE"] = "https://a.fe8.cn/v1"
os.environ["OPENAI_API_KEY"] = "sk-jxGxEstKGNXlwD4gaFIIveFQOAWm7hRxUZth8k191o5m8DE4"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),
                base_url=os.environ.get("OPENAI_API_BASE"),
                timeout=openai.Timeout(600, connect=600))

def get_completion(prompt, model, temperature, message):
    res = ''
    try:
        response = client.chat.completions.create(
            model=model,
            # TODO 这里修改system设定
            messages=[
                {"role": "system",
                 "content": message[0]},
                {"role": "user", "content": prompt},
            ],
            temperature=temperature,
            # max_tokens=4096,
        )
        res = response.choices[0].message.content
        # 响应tokens数量
        prompt_tokens_num = response.usage.prompt_tokens
        output_tokens_num = response.usage.completion_tokens
        return res, prompt_tokens_num, output_tokens_num
    except openai.APIConnectionError as e:
        logging.error(f'The server could not be reached, cause:{e.__cause__}')
    except openai.RateLimitError as e:
        logging.error('A 429 status code was received; we should back off a bit.')
    except openai.APIStatusError as e:
        logging.error(
            f'Another non-200-range status code was received, status_code: {e.status_code}, error response: {e.response}')
    except Exception as e:
        logging.error(
            f'Another error response: {e}')
    return res, 0, 0

In [ ]:
import time


def run_term_extraction(times):
    
    with open('assets/system_description.txt', 'r', encoding='utf-8') as f:
        system_description = f.read()
    
        # chain of thought example
    with open('assets/chain_of_thought_term_extraction.txt', 'r') as f:
        chain_of_thought_term_extraction = f.read()

    
    with open('assets/requirement_architecture.json', 'r') as f:
        requirement_file = f.read()
    
    message = [ 'You are an expert in requirement engineering.' +
            'Requirement statements can be categorised into Functional Requirements and Non-Functional Requirements. ' +
            'You will be provided with some requirement statements from User, which are for a system to be developed.' +
            'The description of the system is here: ' + system_description +
            'Your task is to extract Concepts and their Instances from the requirement statements.' +
            'The requirements will be provided by User, please note that the requirements are not ordered, and one requirement may implicitly depend on other requirements. ' +
            'Each requirement statement is identified by a unique GID (short for Global ID). ' +
            'Please extract occurrences of the following from the requirement statements: "GID", "Concepts" and "Instances". ' +
            'The rationale for your task is here: ' + chain_of_thought_term_extraction +
            'The Output format should be in JSON only, no markdown permitted (no ```), no explain.' +
            'Include only contents provided to you.']
    
    for i in range(times):
        start = time.time()
        model = "gpt-4o"
        temperature = 0.2
        prompt = requirement_file
        res = get_completion(prompt, model=model, temperature=temperature, message=message)

        print(f"Iteration {i+1}:")
        print(f"Time taken: {time.time() - start} seconds")
        print(res[0])  # gpt4的结果，看这里

        with open(f'result/Term_extraction_result/single_term_extraction_result_{i+1}.json', 'w', encoding='utf-8') as f:
            f.write(res[0])

In [ ]:
times = 100
run_term_extraction(times)

In [ ]:
def run_dsl_extraction(times):
    
    with open('assets/system_description.txt', 'r', encoding='utf-8') as system_file:
        system_description = system_file.read()
    
    # Few-shot example for dsl extraction
    with open('assets/few_shot_example_dsl_extraction.txt', 'r', encoding='utf-8') as few_shot_dsl_file:
        few_shot_example_dsl_extraction = few_shot_dsl_file.read()
    
    # chain of thought prompt for dsl extraction
    with open('assets/chain_of_thought_dsl_extraction.txt', 'r') as cot_dsl_extraction:
        chain_of_thought_dsl_extraction = cot_dsl_extraction.read()
    with open("assets/requirement_architecture.json", 'r', encoding='utf-8') as rq:
        requirement_file = rq.read()
  
    for i in range(times):
        
        with open(f'result/Term_extraction_result/single_term_extraction_result_{i+1}.json', 'r', encoding='utf-8') as term_file:
            extracted_terms = term_file.read()
        message = [
            'You are an expert in Model Driven Engineering (MDE). ' +
            'You are experienced in creating "Domain Specific Languages (DSLs)" or "metamodels". ' +
            'A metamodel contains the abstract syntax to capture the concepts for the system to be developed. ' +
            'You have extensive experience in creating metamodels using the Ecore modeling language provided by the Eclipse Modeling Framework (EMF). ' +
            'You are also an expert in creating metamodels using the Emfatic language (a textual syntax for Ecore) to create metamodels.' +
            'Your job is to create a metamodel written in Emfatic based on the requirements provided to you in JSON every round. ' +
            'The description of the system to be developed is here: ' + system_description +
            'Previously, a Term model is extracted, in here: ' + extracted_terms +
            'Here are some thinking processes to guide you: ' + chain_of_thought_dsl_extraction +
            'Here are some examples of the extracted metamodels based on inputs:' + few_shot_example_dsl_extraction +
            'Consider also the previous inputs from the User. ' +
            'You must only include Emfatic code in your responses.' +
            'Do not include any other contents that are not provided to you. ' +
            'Please do not include explanations in your answers. In addition, no markdown is permitted (no ```). ',
            ]   
        start = time.time()
        model = "gpt-4o"
        temperature = 0.2
        prompt = requirement_file
        res = get_completion(prompt, model=model, temperature=temperature, message=message)

        print(f"Iteration {i+1}:")
        print(f"Time taken: {time.time() - start} seconds")
        print(res[0])  # gpt4的结果，看这里

        with open(f'result/DSL_extraction_result/single_DSL_extraction_result_{i+1}.emf', 'w', encoding='utf-8') as f:
            f.write(res[0])


In [ ]:
run_dsl_extraction(times)

In [ ]:
def run_dsl_trace_extraction(times):   
    message = [
    'You are an expert in Model Driven Engineering (MDE). ' +
    'Your job is to generate a traceability JSON element based on the Emfatic code and the requirement in JSON provided by User  ' +
    'In the Emfatic code provided to you by DSL_Summariser, you shall identify which class, attributes, references and operations are created, based on the requirement JSON provided by User' +
    'Your answer should be in the following JSON format : ' +
    '{"requirement_gid" : ${the gid of the requirement provided by User}, ' +
    '"Emfatic_class" : ${the class name provided by Model_Checker}, ' +
    '"Emfatic_attributes": ${the "attr"s that the class contains (if not null)}, ' +
    '"Emfatic_references":${the "ref"s or "val"s that the class contains (if not null)}, ' +
    '"Emfatic_operations": ${the "op"s that class contains (if not null)} }' +
    'If the Emfatic code creates multiple classes, please create a JSON list with multiple JSON entries, like so: [$JSON entry, $JSON entry, ... $JSON entry].' +
    ' The Output format is JSON only, no markdown permitted (no ```), no explain.'
] 
    
    for i in range(times):
        with open("assets/requirement_architecture.json", 'r', encoding='utf-8') as rq:
            requirement_file = rq.read()

        with open(f'result/DSL_extraction_result/single_DSL_extraction_result_{i+1}.emf', 'r', encoding='utf-8') as code:
            emfatic_code = code.read()
        
        start = time.time()
        model = "gpt-4o"
        temperature = 0.2
        prompt = 'This is requirement JSON:'+requirement_file+'this is the Emfatic code: '+emfatic_code
        
        res = get_completion(prompt, model=model, temperature=temperature, message=message)

        print(f"Iteration {i+1}:")
        print(f"Time taken: {time.time() - start} seconds")
        print(res[0])  # gpt4的结果，看这里

        with open(f'result/DSL_trace_result/single_DSL_trace_result_{i+1}.json', 'w', encoding='utf-8') as f:
            f.write(res[0])

In [ ]:
run_dsl_trace_extraction(times)

In [ ]:
def run_emf_creation(times):   
    # Few-shot example for modeller
    with open('assets/few_shot_example_model_creation.txt', 'r', encoding='utf-8') as f:
        few_shot_example_model_creation = f.read()   
    # chain of thought prompt for modeller
    with open('assets/chain_of_thought_model_creation.txt', 'r') as f:
        chain_of_thought_model_creation = f.read()   
    with open("assets/requirement_architecture.json", 'r', encoding='utf-8') as rq:
        requirement_file = rq.read()       
    for i in range(times):
        with open(f'result/Term_extraction_result/single_term_extraction_result_{i+1}.json', 'r') as f:
            term_extraction = f.read()
        with open(f'result/DSL_extraction_result/single_DSL_extraction_result_{i+1}.emf', 'r', encoding='utf-8') as DSL_source:
            dsl = DSL_source.read()
        message = [
            'You are an expert in Model Driven Engineering (MDE). ' +
            'You are experienced in creating EMF (Eclipse Modelling Framework) models using the Epsilon Object Language (EOL). ' +
            'Your job is to write an EOL program to create an EMF model that conforms to the EMF metamodel (written in the Emfatic language) in here: ' + dsl +
            'You should base your answer on the requirements provided to you by User. ' +
            'The Concepts extracted (in JSON) from the requirements are here: ' + term_extraction +
            'Consult the thinking process here: ' + chain_of_thought_model_creation +
            'Here are some examples based on input requirements:' + few_shot_example_model_creation +
            'You should also consult the code produced by Model_Refactor. ' +
            'You must only include EOL code in your responses. ' +
            'Do not include any other contents that are not provided to you. ' +
            'Please do not include explanations in your answers. Do not include markdowns (no ```). ',
            ]
        start = time.time()
        model = "gpt-4o"
        temperature = 0.2
        prompt = requirement_file
        res = get_completion(prompt, model=model, temperature=temperature, message=message)

        print(f"Iteration {i+1}:")
        print(f"Time taken: {time.time() - start} seconds")
        print(res[0])  # gpt4的结果，看这里

        with open(f'result/EMF_Model_result/single_eol_result_{i+1}.eol', 'w', encoding='utf-8') as f:
            f.write(res[0])

In [ ]:
run_emf_creation(times)

In [ ]:
def run_emf_trace_creation(times):
    
    with open('assets/chain_of_thought_model_trace.txt', 'r') as f:
        chain_of_thought_model_trace = f.read()
    
    with open("assets/requirement_architecture.json", 'r', encoding='utf-8') as rq:
        requirement_file = rq.read()  
    for i in range(times):
        # Emfatic rules
        with open(f'result/DSL_extraction_result/single_DSL_extraction_result_{i+1}.emf', 'r', encoding='utf-8') as DSL_source:
            dsl = DSL_source.read()
            
        with open(f'result/EMF_Model_result/single_eol_result_{i+1}.eol', 'r', encoding='utf-8') as code:
            eol_code = code.read()
        
        message = [
        'You are an expert in Model Driven Engineering (MDE). ' +
        'You are experienced in creating EMF (Eclipse Modelling Framework) models using languages provided by the Eclipse Epsilon platform. ' +
        'Specifically, you use the Epsilon Object Language (EOL) to create EMF models programmatically. ' +
        'Your job is to generate a JSON element by analysing the EOL code (provided by Model_Checker) for model creation, and the requirement provided by User every round. ' +
        'The metamodel (created using the Emfatic language) for the model to be created is here: ' + dsl +
        'Here are some guidance to help you: ' + chain_of_thought_model_trace +
        'Your answer should be in the following JSON format : ' +
        '{"requirement_gid" : ${the gid of the requirement provided by User}, ' +
        '"Model_Element" : ${the name of the class created by the EOL code}, ' +
        '"Model_Element_id" : ${the name or the id of the object created for the above class}, ' +
        '"Model_attribute": ${the name of the attributes set by the EOL code}, ' +
        '"Attribute_value": ${the value of the above attribute set by the EOL code}, ' +
        '"Model_references":${the name of the references set by the EOL code} } ' +
        '"Reference_value":${the name or the id of the value set for the above reference} } ' +
        'If the EOL code creates multiple model elements, please create a JSON list with multiple JSON entries, like so: [$JSON entry, $JSON entry, ... $JSON entry].' +
        ' The Output format is JSON only, no markdown permitted (no ```), no explain.']
        
        start = time.time()
        model = "gpt-4o"
        temperature = 0.2
        prompt = 'This is requirement JSON:'+requirement_file+'this is the Emfatic code: '+eol_code
        res = get_completion(prompt, model=model, temperature=temperature,message=message)
        print(f"Iteration {i+1}:")
        print(f"Time taken: {time.time() - start} seconds")
        print(res[0])  # gpt4的结果，看这里
        with open(f'result/EMF_Model_trace_result/single_EOL_trace_result_{i+1}.json', 'w', encoding='utf-8') as f:
            f.write(res[0])

In [ ]:
run_emf_trace_creation(times)